In [1]:
# !pip install keras-tuner
# !pip install nbconvert
import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.applications import VGG16, ResNet50
from keras.models import Sequential
from keras.layers import Conv2D, Lambda, MaxPooling2D, ZeroPadding2D # convolution layers
from keras.layers import Dense, Dropout, Flatten # core layers
from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing import image_dataset_from_directory
from keras.optimizers.legacy import Adam, SGD
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
import keras_tuner
from keras_tuner.tuners import RandomSearch, Hyperband
import torch

In [2]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device = mps_device)
    print(x)
else:
    print ("MPS device not found.")

print(tf.__version__)
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

tensor([1.], device='mps:0')
2.13.0
Default GPU Device: /device:GPU:0


2024-11-03 01:17:49.535837: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-11-03 01:17:49.535856: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-11-03 01:17:49.535859: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-11-03 01:17:49.535887: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-03 01:17:49.535900: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-11-03 01:17:49.536322: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been buil

In [3]:
# Define constants
img_width, img_height = 256, 256
batch_size = 21
epochs = 30
num_classes = 3

# Define the directory where your data is stored
data_path = '../Desktop/Stuff/Tea Photos/Complete Directory'

# Use data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2 # Splitting the data into training and validation sets
)

# Generate training dataset
train_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Specify the subset as training data
)

# Generate validation dataset
validation_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Specify the subset as validation data
)

train_img, train_lables = train_generator.next()
validation_img, validation_lables = validation_generator.next()

train_lables = train_lables.reshape(-1, num_classes)
validation_lables = validation_lables.reshape(-1, num_classes)

Found 168 images belonging to 3 classes.
Found 42 images belonging to 3 classes.


In [4]:
# Load the pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the pre-trained layers so they are not trained during the Weather detection process
for layer in base_model.layers:
    layer.trainable = False

# Define the model-building function for Keras Tuner
def build_model(hp):
    model = Sequential()
    model.add(base_model) # output.shape = (8, 8, 512)

    # Choose the number of conv layers
    filters = hp.Choice('filters', values=[512, 1024])
    for i in range(hp.Int('num_conv_layers', min_value=2, max_value=4)):
        model.add(ZeroPadding2D(padding=(1, 1)))
        model.add(Conv2D(filters=filters, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(2, 2)) # output.shape = (4, 4, 512/1024)

    model.add(Flatten())

    # Choose the number of dense layers
    for j in range(hp.Int('num_dense_layers', min_value=1, max_value=3)):
        model.add(Dense(units=hp.Choice(f'units_{j}', values = [16, 128, 1024]), activation='relu'))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

2024-11-03 01:17:52.639254: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-03 01:17:52.639271: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
# Initialize the Keras Tuner Hyperband
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=180,
    executions_per_trial=1,
    directory='my_test_2',
    project_name='hyperparameter_tuning_1'
)

# Perform the hyperparameter search
tuner.search(train_img, train_lables, epochs=epochs, validation_data=(validation_img, validation_lables), batch_size = batch_size)

# Get the best hyperparameters
best_hyperparameters = tuner.oracle.get_best_trials(1)[0].hyperparameters.values
best_num_conv_layers = best_hyperparameters['num_conv_layers']
best_num_dense_layers = best_hyperparameters['num_dense_layers']

# Print the best hyperparameters
print(f"Best Number of Convolutional Layers: {best_num_conv_layers}")
print(f"Best Number of Dense Layers: {best_num_dense_layers}")

# Build the final model with the best hyperparameters
best_trial = tuner.oracle.get_best_trials(1)[0]
best_model = tuner.hypermodel.build(best_trial.hyperparameters)
# best_model.summary()

tuner.results_summary()

Reloading Tuner from my_test_2/hyperparameter_tuning_1/tuner0.json
Best Number of Convolutional Layers: 2
Best Number of Dense Layers: 3
Results summary
Results in my_test_2/hyperparameter_tuning_1
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 094 summary
Hyperparameters:
filters: 1024
num_conv_layers: 2
num_dense_layers: 3
units_0: 128
learning_rate: 0.01
units_1: 16
units_2: 16
Score: 0.7142857313156128

Trial 119 summary
Hyperparameters:
filters: 512
num_conv_layers: 3
num_dense_layers: 2
units_0: 128
learning_rate: 0.001
units_1: 128
units_2: 1024
Score: 0.7142857313156128

Trial 167 summary
Hyperparameters:
filters: 1024
num_conv_layers: 2
num_dense_layers: 3
units_0: 128
learning_rate: 0.001
units_1: 1024
units_2: 16
Score: 0.7142857313156128

Trial 092 summary
Hyperparameters:
filters: 512
num_conv_layers: 4
num_dense_layers: 3
units_0: 128
learning_rate: 0.001
units_1: 128
units_2: 128
Score: 0.6666666865348816

Trial 035 summary
Hyperparameters:

In [6]:
best_model.save('tea_classifier.keras')
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 zero_padding2d (ZeroPaddin  (None, 10, 10, 512)       0         
 g2D)                                                            
                                                                 
 conv2d (Conv2D)             (None, 8, 8, 1024)        4719616   
                                                                 
 zero_padding2d_1 (ZeroPadd  (None, 10, 10, 1024)      0         
 ing2D)                                                          
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 1024)        9438208   
                                                                 
 max_pooling2d (MaxPooling2  (None, 4, 4, 1024)        0